In [1]:
from astroquery.gaia import Gaia

import numpy as np
import pandas as pd
import sys
sys.path.append('/Users/jinzhao/Documents/GitHub/2ES_target_selection/scripts/')
from query import *

In [2]:
# Query parameters
TARGET_G_MAG_LIMIT = 12  # gs.phot_g_mean_mag < TARGET_G_MAG_LIMIT
NEIGHBOR_G_MAG_LIMIT = 15  # neighbors.phot_g_mean_mag < NEIGHBOR_G_MAG_LIMIT
MIN_DEC = -85  # gs.dec >= MIN_DEC
MAX_DEC = 30  # gs.dec <= MAX_DEC
MIN_PARALLAX = 20  # gs.parallax >= MIN_PARALLAX (MAX_DISTANCE = 1000/MIN_PARALLAX)

In [3]:
query_all_stars = f"""
SELECT gs.source_id AS dr2_source_id,
    gs.ra, 
    gs.dec, 
    gs.phot_g_mean_mag, 
    gs.phot_bp_mean_mag, 
    gs.phot_rp_mean_mag, 
    gs.bp_rp, 
    gs.parallax
FROM gaiadr2.gaia_source AS gs
WHERE gs.phot_g_mean_mag < {TARGET_G_MAG_LIMIT}
    AND gs.dec BETWEEN {MIN_DEC} AND {MAX_DEC}
    AND gs.parallax >= {MIN_PARALLAX} 
"""

no_bright_neighbor_query = f"""
SELECT 
    gs.source_id, 
    gs.ra, 
    gs.dec, 
    gs.phot_g_mean_mag
FROM 
    gaiadr2.gaia_source AS gs
WHERE 
    gs.phot_g_mean_mag < {TARGET_G_MAG_LIMIT}
    AND gs.dec BETWEEN {MIN_DEC} AND {MAX_DEC}
    AND gs.parallax >= {MIN_PARALLAX}
    AND NOT EXISTS (
        SELECT 1
        FROM gaiadr2.gaia_source AS neighbors
        WHERE 
            1=CONTAINS(
                POINT('ICRS', gs.ra, gs.dec),
                CIRCLE('ICRS', neighbors.ra, neighbors.dec, 2/3600.0)
            )
            AND neighbors.phot_g_mean_mag < {NEIGHBOR_G_MAG_LIMIT}
            AND gs.source_id != neighbors.source_id
    )
"""

bright_neighbor_query = f"""
SELECT 
    gs.source_id, 
    gs.ra, 
    gs.dec, 
    gs.phot_g_mean_mag, 
    neighbors.source_id AS neighbor_id,
    neighbors.phot_g_mean_mag AS neighbor_mag,
    DISTANCE(
        POINT('ICRS', gs.ra, gs.dec), 
        POINT('ICRS', neighbors.ra, neighbors.dec)
    ) AS angular_distance
FROM 
    gaiadr2.gaia_source AS gs
JOIN 
    gaiadr2.gaia_source AS neighbors
ON 
    1=CONTAINS(
        POINT('ICRS', gs.ra, gs.dec),
        CIRCLE('ICRS', neighbors.ra, neighbors.dec, 2/3600.0)
    )
WHERE 
    gs.phot_g_mean_mag < {TARGET_G_MAG_LIMIT}
    AND gs.dec BETWEEN {MIN_DEC} AND {MAX_DEC}
    AND gs.parallax >= {MIN_PARALLAX}
    AND neighbors.phot_g_mean_mag < {NEIGHBOR_G_MAG_LIMIT}
    AND gs.source_id != neighbors.source_id
"""

# ... rest of the code remains the same ...

In [4]:


if 0: # For the all stars query
    df_all_stars = execute_gaia_query(
        query_all_stars,
        str_columns=['dr2_source_id'],
        output_file='all_stars_dr2.xlsx'
    )

if 1: # For the no bright neighbor query
    df_no_bright_neighbor = execute_gaia_query(
        no_bright_neighbor_query,
        str_columns=['source_id'],
        output_file='no_bright_neighbor_results.xlsx'
    )

if 0: # For the bright neighbor query
    df_bright_neighbor = execute_gaia_query(
        bright_neighbor_query,
        str_columns=['source_id', 'neighbor_id'],
        output_file='bright_neighbor_results.xlsx'
    )

An error occurred: [Errno 60] Operation timed out
